In [1]:
import os, sys, json
sys.path.append("/workspace/AutoCog")
from autocog.language.sta.compile import Compiler
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
sources = ["""
prompt mmlu_main {
    is {
        topic is text;
        question is text;
        choices[4] is {
            value is text;
            correct is enum("yes","no");
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}""","""
format bool {
    is enum("true","false");
    annotate "a bolean value is either true or false";
}
struct a_choice {
    is {
        value   is text;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text<20>;
        question   is text<length=50>;
        choices[4] is a_choice;
        answer     is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
"""]

In [3]:
compiler = Compiler(source=sources[1])
wrap_graphviz(compiler.program.toGraphViz())

NotImplementedError: Instantiate struct:
	name='a_choice' arguments=[]